<a href="https://colab.research.google.com/github/lucasnserra/CustomerSegmentation/blob/main/Time_Cohorts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###El Conjunto de datos es una muestra aleatoria compuesta por medio millon de transacciones de datos minoristas en linea




In [23]:
#Importo Librerias
import numpy as np
import pandas as pd
import datetime as dt

In [24]:
#Cargo el dataset del Raw en git y determino como columna de fechas InvoiceDate para que datetime funcione
online = pd.read_csv('https://raw.githubusercontent.com/lucasnserra/CustomerSegmentation/main/online.csv',parse_dates=['InvoiceDate'])
online.head()
## Las columnas principales son InvoiceDate, UnitPrice y CustomerID

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom


In [25]:
 #Creo funcion para obtener primer dia del mes
def get_day(x): 
  return dt.datetime(x.year,x.month,x.day) #return dt.datetime(x.year,x.month,1) 

In [26]:
#Creo la comlumna de día 
online['InvoiceDay'] = online['InvoiceDate'].apply(get_day)
#agrupo InvoiceDay por CostumerID 
grouping = online.groupby('CustomerID')['InvoiceDay']
#Le asigno el minimo a cada cliente de su compra
online['CohortDay'] = grouping.transform('min') 
online.head(5)

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,CohortDay
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,2011-10-25,2011-04-11
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,2011-11-20,2011-09-12
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,2011-07-14,2011-07-14
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2011-11-23,2011-11-23
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,2011-05-25,2011-02-25


In [27]:
#Creo funcion para calcular el tiempo entre la fecha de alta y la fecha de la transaccion 
def get_date_int(df,column):
  year = df[column].dt.year
  month = df[column].dt.month
  day = df[column].dt.day
  return year,month,day

In [28]:
#Extraigo componentes de la fecha de la factura 
invoice_year, invoice_month, invoice_day = get_date_int(online,'InvoiceDate')
#Extraigo componentes de la fecha de cohort 
cohort_year, cohort_month, cohort_day = get_date_int(online,'CohortDay')

In [29]:
#Calculo diferencia entre años
years_diff = invoice_year - cohort_year
#Calculo diferencia entre Meses
months_diff = invoice_month - cohort_month
#Calculo diferencia entre Dias
days_diff = invoice_day - cohort_day
#Creo Index en dataset con la suma de la cantidad de dias calculados previamente
online['CohortIndex'] = years_diff * 365 + months_diff * 12 + days_diff + 1  # Se suma 1 al final para que no quede 0 la resta si la fecha de invoice es igual a la de cohort
online.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,CohortDay,CohortIndex
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,2011-10-25,2011-04-11,87
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,2011-11-20,2011-09-12,33
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,2011-07-14,2011-07-14,1
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2011-11-23,2011-11-23,1
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,2011-05-25,2011-02-25,37


In [30]:
# Creo una tabla pivotante para corroborar como queda el Cohort por tiempo // Se comenta ya que se debe crear sobre meses no sobre dias
#grouping = online.groupby(['CohortDay','CohortIndex'])
#cohort_data = grouping['CustomerID'].apply(pd.Series.nunique)
#cohort_data = cohort_data.reset_index()
#cohort_counts = cohort_data.pivot(index = 'CohortDay',columns = 'CohortIndex',values= 'CustomerID')
#cohort_counts